In [1]:
import pandas as pd
import numpy as np
import min_features, daily_return
import importlib
importlib.reload(min_features)
importlib.reload(daily_return)

perf_df = pd.read_csv("h2_performance.csv")
#perf_df = perf_df.drop_duplicates()
perf_df['Date'] = perf_df['test_start']
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feats = daily_return.pull_daily('QQQ', returns) 
return_cols = df_daily.columns[df_daily.columns.str.contains("Return_")].to_list()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, log_loss, matthews_corrcoef, balanced_accuracy_score
import pandas as pd
import numpy as np
import min_features, daily_return
import importlib
importlib.reload(min_features)
importlib.reload(daily_return)

perf_df = pd.read_csv("h2_performance.csv")
#perf_df = perf_df.drop_duplicates()
perf_df['Date'] = perf_df['test_start']
returns = [1, 2, 3, 5, 10, 20, 30]
#df_daily, feats = daily_return.pull_daily('QQQ', returns) 
#return_cols = df_daily.columns[df_daily.columns.str.contains("Return_")].to_list()

perf_df = pd.read_csv("h2_performance.csv")
perf_df = perf_df.drop_duplicates()
gcols = ["horizon", "model", "train_years", "feature_set", "pi_size", "min_feats"]

y_col = "test_pos_n"   # 0/1 actual
p_col = "pred"         # P(y=1)

def _clip01(p, eps=1e-15):
    p = np.asarray(p, dtype=float)
    return np.clip(p, eps, 1 - eps)

def _metrics(min_th, cov_th, g: pd.DataFrame) -> pd.Series:
    y = g[y_col].astype(int).to_numpy()
    p = _clip01(g[p_col].to_numpy())

    yhat = np.where(p >= min_th, 1, np.where(p <= (1 - min_th), 0, np.nan))

    # keep only confident predictions (>=min_th in either direction)
    conf = ~np.isnan(yhat)
    y = y[conf]
    p = p[conf]
    yhat = yhat[conf].astype(int)
    records = int(len(y))

    # confident subset mask
    sel = (p >= cov_th) | (p <= (1-cov_th))

    # probability metrics
    brier = brier_score_loss(y, p)
    ll = log_loss(y, p)

    # MCC (needs both classes in y and yhat)
    mcc = np.nan
    if (np.unique(y).size > 1) and (np.unique(yhat).size > 1):
        mcc = matthews_corrcoef(y, yhat)

    if np.unique(y).size > 1:
        bal_acc = balanced_accuracy_score(y, yhat)

        # Positive class accuracy (TPR)
        pos_mask = (y == 1)
        if pos_mask.any():
            pos_acc = float((yhat[pos_mask] == 1).mean())

        # Negative class accuracy (TNR)
        neg_mask = (y == 0)
        if neg_mask.any():
            neg_acc = float((yhat[neg_mask] == 0).mean())

    # confident accuracy + coverage
    cov = float(sel.mean())
    acc_conf = float((yhat[sel] == y[sel]).mean()) if sel.any() else np.nan
    
    prec_pos_all = np.nan
    prec_neg_all = np.nan
    prec_pos_th = np.nan
    prec_neg_th = np.nan

    # positive-class precision
    pred_pos = (yhat == 1)
    if pred_pos.any():
        prec_pos_all = float((y[pred_pos] == 1).mean())   # TP/(TP+FP)

    pred_neg = (yhat == 0)
    if pred_neg.any():
        prec_neg_all = float((y[pred_neg] == 0).mean())   # TN/(TN+FN)

    # positive-class precision
    pred_pos = (yhat == 1) & sel
    if pred_pos.any():
        prec_pos_th = float((y[pred_pos] == 1).mean())

    # negative-class precision
    pred_neg = (yhat == 0) & sel
    if pred_neg.any():
        prec_neg_th = float((y[pred_neg] == 0).mean())

    return pd.Series({
        "pos_rate": float(y.mean()),
        "records": records,
        "bal_prec": round((prec_pos_all + prec_neg_all) / 2, 2),
        f"bal_prec_|{cov_th}|": round((prec_pos_th + prec_neg_th) / 2, 2),
        "brier": float(brier),
        "log_loss": float(ll),
        "mcc": float(mcc) if not np.isnan(mcc) else np.nan,
        "pprec": prec_pos_all, 
        "nprec": prec_neg_all,
        f"cov_|{cov_th}|": cov,
        f"pprec_|{cov_th}|": prec_pos_th,
        f"nprec_|{cov_th}|": prec_neg_th,
    })

min_th = .51
cov_th = .75
pred_df = perf_df[(perf_df["pred"] > min_th) | (perf_df["pred"] < (1 - min_th))].copy()
metrics_df = (
    pred_df
    .dropna(subset=[y_col, p_col])
    .groupby(gcols, sort=False)
    .apply(lambda g: _metrics(min_th, cov_th, g), include_groups=False)
    .reset_index()
    .sort_values(["horizon", "mcc", "brier"], ascending=[True, False, True])
)

metrics_df['composite'] =  0.5*(metrics_df['bal_prec']) + 0.25*(metrics_df['mcc']) + .125*(1-(metrics_df['brier'])) + 0.1*((metrics_df[f'bal_prec_|{cov_th}|']) * (metrics_df[f'cov_|{cov_th}|']))
# top per horizon (ranked by MCC desc, then Brier asc)
top_by_horizon = (
    metrics_df
    .sort_values(["horizon", "composite", "log_loss"], ascending=[True, False, False])
    .groupby("horizon", as_index=False, sort=False)
    .head(150)
)

#top_by_horizon.to_csv('top_performers.csv', index=False)
#top_by_horizon[top_by_horizon['horizon'] == 2].round(2)
#top_by_horizon[top_by_horizon['feature_set'] == 'ma_sma-atr_adxvola'].round(2)
top_by_horizon.round(2)

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
19,2,xgboost-3,3,ma_sma-volu,1.5,4,0.57,147.0,0.58,0.59,0.34,5.52,0.16,0.63,0.54,0.85,0.62,0.55,0.46
38,2,xgboost-3,3,volu-past_return,1.5,4,0.58,149.0,0.57,0.59,0.33,4.37,0.13,0.63,0.50,0.81,0.62,0.56,0.45
27,2,xgboost-3,3,ma_num-vix_skew,1.5,4,0.58,146.0,0.57,0.54,0.35,5.16,0.13,0.63,0.51,0.82,0.64,0.44,0.44
6,2,xgboost-3,3,ma_lag-vix_skew,1.5,4,0.57,146.0,0.55,0.57,0.34,4.90,0.11,0.61,0.50,0.79,0.61,0.54,0.43
37,2,xgboost-3,3,volu-experimental_slope,1.5,4,0.59,147.0,0.54,0.54,0.37,5.18,0.07,0.61,0.46,0.82,0.60,0.48,0.41
41,2,xgboost-3,3,atr_adxvola-past_return,1.5,4,0.56,144.0,0.54,0.55,0.36,4.56,0.07,0.59,0.48,0.76,0.57,0.52,0.41
8,2,xgboost-3,3,ma_lag-past_return,1.5,4,0.58,146.0,0.53,0.55,0.35,4.48,0.07,0.60,0.47,0.77,0.59,0.51,0.41
50,2,xgboost-3,3,volu,1.5,4,0.57,148.0,0.53,0.51,0.40,6.34,0.07,0.59,0.47,0.86,0.56,0.45,0.40
43,2,xgboost-3,3,vix_skew-past_return,1.5,4,0.57,146.0,0.53,0.50,0.36,4.95,0.07,0.60,0.47,0.73,0.60,0.41,0.40
1,2,xgboost-3,3,ma_lag-ma_sma,1.5,4,0.57,147.0,0.53,0.53,0.38,7.15,0.05,0.59,0.46,0.80,0.59,0.47,0.40


In [591]:
for r in returns:
    perf_df_ph = perf_df[(perf_df['horizon'] == r)].drop_duplicates().copy()
    perf_df_ph.to_csv(f"horizon_{r}_baseline.csv", index=False)

In [ ]:
perf_df_2 = perf_df[(perf_df['horizon'] == 2) & (perf_df['model'] == 'xgboost-3') 
                    & (perf_df['train_years'] == 6) & (perf_df['feature_set'] == 'ma_rel_ba')
                    &(perf_df['pi_size'] == 1.5) & (perf_df['min_feats'] == 100)].drop_duplicates().copy()
print(len(perf_df_2))
perf_df_5 = perf_df[(perf_df['horizon'] == 5) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_.001_<10_.75_nopr')].drop_duplicates().copy()
print(len(perf_df_5))
perf_df_10 = perf_df[(perf_df['horizon'] == 10) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_mirror')].drop_duplicates().copy()
print(len(perf_df_10))
perf_df_20 = perf_df[(perf_df['horizon'] == 20) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 6) & (perf_df['feature_set'] == 'pi_.001_<10_.75_pr')].drop_duplicates().copy()
print(len(perf_df_20))
perf_df_30 = perf_df[(perf_df['horizon'] == 30) & (perf_df['model'] == 'xgboost-2') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_base')].drop_duplicates().copy()
print(len(perf_df_30))

In [ ]:
perf_df_2 = perf_df[(perf_df['horizon'] == 20) & (perf_df['model'] == 'xgboost-3') 
                    & (perf_df['train_years'] == 6) & (perf_df['feature_set'] == 'ma_rel_ba')
                    &(perf_df['pi_size'] == 1.5) & (perf_df['min_feats'] == 100)].drop_duplicates().copy()
print(len(perf_df_2))
perf_df_5 = perf_df[(perf_df['horizon'] == 5) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_.001_<10_.75_nopr')].drop_duplicates().copy()
print(len(perf_df_5))
perf_df_10 = perf_df[(perf_df['horizon'] == 10) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_mirror')].drop_duplicates().copy()
print(len(perf_df_10))
perf_df_20 = perf_df[(perf_df['horizon'] == 20) & (perf_df['model'] == 'xgboost-6') 
                    & (perf_df['train_years'] == 6) & (perf_df['feature_set'] == 'pi_.001_<10_.75_pr')].drop_duplicates().copy()
print(len(perf_df_20))
perf_df_30 = perf_df[(perf_df['horizon'] == 30) & (perf_df['model'] == 'xgboost-2') 
                    & (perf_df['train_years'] == 5) & (perf_df['feature_set'] == 'pi_base')].drop_duplicates().copy()
print(len(perf_df_30))
#best_models_perf = pd.concat([perf_df_2, perf_df_5, perf_df_10, perf_df_20, perf_df_30], ignore_index=True)
#best_models_perf.to_csv("best_models_perf.csv", index=False)

0
0
0
0
0


In [ ]:
def flip_bucket_tables_multi_dual(
    df_daily,
    perf_df,
    returns,
    *,
    K=3,
    date_col="Date",
    close_col="Close",
    w=None,
    perf_filter=None,  # optional callable to filter perf_df per horizon
):
    """
    For each horizon r:
      - builds streak + streak_lag1 from Return_r
      - merges into perf_df rows for (horizon=r) (and any extra filters you provide)
      - buckets BOTH streak and streak_lag1 into [-K..K] plus tails as +/- (K+1) labeled "3+"
      - computes:
          - wba_close (from streak) and wba_open (from streak_lag1)
          - bal_acc pair scores for +/-1, +/-2, +/-3 for both contexts
          - (optional) keeps acc/n wide columns for each context with suffixes _c and _o

    Returns:
      by_r: dict[r] -> wide table (flat columns)
      all_out: concat of all horizons with horizon as index level (flat columns)
    """
    if w is None:
        # weights: ±1 -> 2.0, ±2 -> 1.5, ±3 -> 1.25, ±3+ -> 1.0
        w = {1: 2.0, 2: 1.5, 3: 1.25, "3+": 1.0}

    max_score = float(sum(w.values()))
    gcols = ["model", "train_years", "feature_set", "pi_size", "min_feats"]

    def _add_streak(df_base, ret_col):
        d = df_base[[date_col, close_col, ret_col]].sort_values(date_col).copy()
        s = d[ret_col].astype("int8")
        grp = s.ne(s.shift()).cumsum()
        streak_len = s.groupby(grp).cumcount() + 1
        d["streak"] = streak_len.where(s.eq(1), -streak_len).astype("int32")
        d["streak_lag1"] = d["streak"].shift(1).fillna(0).astype("Int64")
        return d

    def _bucketize(series: pd.Series) -> pd.Series:
        b = series.clip(lower=-K, upper=K).astype("int32")
        b = b.copy()
        b.loc[series < -K] = -(K + 1)
        b.loc[series >  K] =  (K + 1)
        return b

    def _make_context_table(d: pd.DataFrame, bucket_col: str, suffix: str) -> pd.DataFrame:
        """
        suffix: "_c" for streak (close), "_o" for streak_lag1 (open)
        returns a flat-column wide df indexed by gcols, containing:
          - acc_{bucket}{suffix}, n_{bucket}{suffix} for buckets {1,-1,2,-2,3,-3,3+,-3+}
          - bal_acc_{1/2/3}{suffix}
          - wba_{close/open} (handled outside)
        """
        flip_perf = (
            d.groupby(gcols + [bucket_col], sort=False)
             .agg(n=("acc", "size"), acc=("acc", "mean"))
        )

        wide = pd.concat(
            {"acc": flip_perf["acc"].unstack(bucket_col),
             "n":   flip_perf["n"].unstack(bucket_col)},
            axis=1
        )

        # enforce order: +1/-1, +2/-2, +3/-3, 3+/-3+
        ordered_cols = []
        for k in [1, 2, 3, "3+"]:
            pb = (K + 1) if k == "3+" else k
            nb = -(K + 1) if k == "3+" else -k
            ordered_cols += [("acc", pb), ("acc", nb), ("n", pb), ("n", nb)]
        wide = wide.reindex(columns=pd.MultiIndex.from_tuples(ordered_cols))

        # relabel bucket keys to strings ("3+", "-3+", etc.)
        rename_cols = []
        for metric, b in wide.columns:
            if b == (K + 1): lab = "3+"
            elif b == -(K + 1): lab = "-3+"
            else: lab = str(b)
            rename_cols.append((metric, lab))
        wide.columns = pd.MultiIndex.from_tuples(rename_cols)

        # flatten to single-level names: acc_1_c, n_-2_o, etc.
        flat = wide.copy()
        flat.columns = [f"{m}_{b}{suffix}" for (m, b) in flat.columns]

        # pair bal_acc for +/-1,2,3 (no 3+ requested here)
        def _pair(acc_pos, acc_neg):
            return (acc_pos + acc_neg) / 2

        flat[f"bal_acc_1{suffix}"] = _pair(flat.get(f"acc_1{suffix}"),  flat.get(f"acc_-1{suffix}"))
        flat[f"bal_acc_2{suffix}"] = _pair(flat.get(f"acc_2{suffix}"),  flat.get(f"acc_-2{suffix}"))
        flat[f"bal_acc_3{suffix}"] = _pair(flat.get(f"acc_3{suffix}"),  flat.get(f"acc_-3{suffix}"))
        flat[f"bal_acc_3p{suffix}"] = _pair(flat.get(f"acc_3+{suffix}"), flat.get(f"acc_-3+{suffix}"))

        return flat

    base = df_daily[[date_col, close_col] + [f"Return_{r}" for r in returns]].copy()

    by_r = {}

    for r in returns:
        ret_col = f"Return_{r}"
        df_r = _add_streak(base, ret_col)

        # merge with perf
        perf_r = perf_df[perf_df["horizon"] == r]
        if perf_filter is not None:
            perf_r = perf_filter(perf_r)

        d = df_r.merge(perf_r, on=date_col, how="inner")

        d["y"] = d[ret_col].astype("int8")
        d["p"] = d["pred"].astype(float)
        d["yhat"] = np.where(
        d["p"] >= min_th, 1,
        np.where(d["p"] <= (1 - min_th), 0, np.nan)
        )

        d = d.dropna(subset=["yhat"]).copy()

        other = perf_r.copy()
        # bucket both contexts
        d["bucket_c"] = _bucketize(d["streak"])       # close-context
        d["bucket_o"] = _bucketize(d["streak_lag1"])  # open-context
        
        #other = d[['Date', "bucket_c"]]

        tab_c = _make_context_table(d, "bucket_c", "_c")
        tab_o = _make_context_table(d, "bucket_o", "_o")

        # combine side-by-side
        out = tab_c.join(tab_o, how="outer")

        # compute wba_close / wba_open from each context’s pair balances
        out["wba_close"] = (
            w[1]   * out["bal_acc_1_c"] +
            w[2]   * out["bal_acc_2_c"] +
            w[3]   * out["bal_acc_3_c"] +
            w["3+"] * out["bal_acc_3p_c"]
        ) / max_score

        out["wba_open"] = (
            w[1]   * out["bal_acc_1_o"] +
            w[2]   * out["bal_acc_2_o"] +
            w[3]   * out["bal_acc_3_o"] +
            w["3+"] * out["bal_acc_3p_o"]
        ) / max_score

        out["wba_close"] = out["wba_close"].round(2)
        out["wba_open"]  = out["wba_open"].round(2)

        # optional: keep horizon as a column too (handy for later concat)
        out = out.reset_index()
        out.insert(0, "horizon", r)

        by_r[r] = out

    all_out = pd.concat(by_r.values(), ignore_index=True)
    return by_r, all_out, other

# ---- usage ----
returns = [2, 5, 10, 20, 30]
by_r, all_out, d = flip_bucket_tables_multi_dual(
    df_daily=df_daily,
    perf_df=perf_df,
    returns=returns,
    K=3,
)

# horizon 10 table
by_r[10].sort_values(["wba_close", "wba_open"], ascending=False).head(25)

# all horizons combined
perf_columns = ['horizon', 'model', 'train_years', 'feature_set', 'pi_size', 'min_feats',
                'n_1_c', 'n_-1_c', 'acc_1_c', 'acc_-1_c',
                 'bal_acc_1_o', 'bal_acc_2_o', 'bal_acc_3_o', 'bal_acc_3p_o', 'wba_open', 'wba_close']

# top per horizon (ranked by MCC desc, then Brier asc)
top_by_horizon = (
    all_out
    .sort_values(["horizon", 'wba_close', 'wba_open'], ascending=[True, False, False])
    .groupby("horizon", as_index=False, sort=False)
    .head(2)
)

top_by_horizon[perf_columns].round(2)

2 | 2026-02-02
5 | 2026-01-28
10 | 2026-01-21
20 | 2026-01-06
30 | 2025-12-19


,horizon,model,train_years,feature_set,pi_size,min_feats,n_1_c,n_-1_c,acc_1_c,acc_-1_c,bal_acc_1_o,bal_acc_2_o,bal_acc_3_o,bal_acc_3p_o,wba_open,wba_close
35,2,xgboost-3,6,experimental_slope_ba,1.5,6,16,16,0.81,0.38,0.63,0.57,0.63,0.57,0.60,0.58
55,2,xgboost-3,6,rsi_macd_ba,1.5,100,16,15,0.81,0.47,0.62,0.58,0.57,0.53,0.58,0.58
70,5,xgboost-3,4,ma_lag_ba,1.5,6,12,13,0.92,0.38,0.55,0.66,0.67,0.68,0.63,0.63
103,5,xgboost-3,6,ma_num-baseline,1.5,8,24,24,0.67,0.50,0.65,0.65,0.59,0.64,0.63,0.63
138,10,xgboost-3,4,ma_rel_ba,1.5,6,9,10,0.89,0.50,0.61,0.62,0.71,0.63,0.64,0.62
165,10,xgboost-3,6,ma_num-baseline,1.5,8,18,20,0.89,0.30,0.59,0.67,0.53,0.50,0.58,0.57
234,20,xgboost-3,6,ma_sma_ba,1.5,6,6,7,0.33,0.57,0.85,0.71,0.88,0.64,0.78,0.71
233,20,xgboost-3,6,ma_sma-baseline,1.5,8,12,14,0.50,0.57,0.69,0.71,0.62,0.72,0.69,0.67
296,30,xgboost-3,6,ma_sma_ba,1.5,6,4,5,0.25,0.80,0.75,0.83,0.83,0.68,0.78,0.73
291,30,xgboost-3,6,ma_num_ba,1.5,100,4,5,1.00,0.40,0.88,0.71,0.50,0.58,0.70,0.72


In [19]:
def calibration_table(g, n_bins=8):
    y = g[y_col].astype(int).to_numpy()
    p = np.clip(g[p_col].to_numpy(dtype=float), 1e-15, 1-1e-15)

    # quantile bins (stable counts); duplicates="drop" prevents errors if probs repeat
    bins = pd.qcut(p, q=n_bins, duplicates="drop")

    out = (
        pd.DataFrame({"bin": bins, "y": y, "p": p})
        .groupby("bin", observed=True)
        .agg(
            n=("y", "size"),
            p_mean=("p", "mean"),   # predicted probability avg
            y_rate=("y", "mean"),   # observed frequency
            p_min=("p", "min"),
            p_max=("p", "max"),
        )
        .reset_index(drop=True)
        .sort_values("p_mean")
    )
    return out

cols = ['horizon']
# example: get calibration table for ONE group
key = (2)  # change
g = perf_df.set_index(cols).loc[key].reset_index()
calib_tbl = calibration_table(g, n_bins=10)
round(calib_tbl,2)

,n,p_mean,y_rate,p_min,p_max
0,317,0.09,0.43,0.00,0.20
1,334,0.35,0.39,0.25,0.40
2,184,0.45,0.46,0.45,0.45
3,447,0.52,0.53,0.50,0.55
4,235,0.60,0.69,0.60,0.60
5,228,0.65,0.66,0.65,0.65
6,194,0.70,0.67,0.70,0.70
7,272,0.79,0.71,0.75,0.85
8,525,0.97,0.71,0.90,1.00


In [ ]:
def calibration_table(g, n_bins=8):
    y = g[y_col].astype(int).to_numpy()
    p = np.clip(g[p_col].to_numpy(dtype=float), 1e-15, 1-1e-15)

    # quantile bins (stable counts); duplicates="drop" prevents errors if probs repeat
    bins = pd.qcut(p, q=n_bins, duplicates="drop")

    out = (
        pd.DataFrame({"bin": bins, "y": y, "p": p})
        .groupby("bin", observed=True)
        .agg(
            n=("y", "size"),
            p_mean=("p", "mean"),   # predicted probability avg
            y_rate=("y", "mean"),   # observed frequency
            p_min=("p", "min"),
            p_max=("p", "max"),
        )
        .reset_index(drop=True)
        .sort_values("p_mean")
    )
    return out

# example: get calibration table for ONE group
#key = (10, "xgboost", 6, "daily")  # change
g = perf_df.set_index(gcols).loc[key].reset_index()
calib_tbl = calibration_table(g, n_bins=20)
round(calib_tbl,2)